In [1]:
import tensorflow as tf
import numpy as np
import time
import os

In [2]:
text = open('khayyam&moulaviSample.txt', 'rb').read().decode(encoding='utf-8')

In [3]:
text[:10]

'|برخیز بتا'

In [4]:
vocabolaries = sorted(set(text))

In [5]:
vocabolaries

['\n',
 ' ',
 '!',
 '|',
 '\xa0',
 '،',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ة',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ٌ',
 'ٍ',
 'َ',
 'ُ',
 'ِ',
 'ّ',
 'ٔ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ۀ',
 'ی']

In [6]:
len(vocabolaries)

53

In [7]:
char2index = {u:i for i, u in enumerate(vocabolaries)}
index2char = np.array(vocabolaries)

In [8]:
char2index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '|': 3,
 '\xa0': 4,
 '،': 5,
 '؟': 6,
 'ء': 7,
 'آ': 8,
 'أ': 9,
 'ؤ': 10,
 'ئ': 11,
 'ا': 12,
 'ب': 13,
 'ة': 14,
 'ت': 15,
 'ث': 16,
 'ج': 17,
 'ح': 18,
 'خ': 19,
 'د': 20,
 'ذ': 21,
 'ر': 22,
 'ز': 23,
 'س': 24,
 'ش': 25,
 'ص': 26,
 'ض': 27,
 'ط': 28,
 'ظ': 29,
 'ع': 30,
 'غ': 31,
 'ف': 32,
 'ق': 33,
 'ل': 34,
 'م': 35,
 'ن': 36,
 'ه': 37,
 'و': 38,
 'ٌ': 39,
 'ٍ': 40,
 'َ': 41,
 'ُ': 42,
 'ِ': 43,
 'ّ': 44,
 'ٔ': 45,
 'پ': 46,
 'چ': 47,
 'ژ': 48,
 'ک': 49,
 'گ': 50,
 'ۀ': 51,
 'ی': 52}

In [9]:
index2char[50]

'گ'

In [10]:
text_as_integer = np.array([char2index[c] for c in text])

In [11]:
text_as_integer

array([ 3, 13, 22, ..., 38, 31, 12])

In [12]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_integer)

In [13]:
char_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [14]:
for i in char_dataset.take(10):
    print(index2char[i.numpy()])

|
ب
ر
خ
ی
ز
 
ب
ت
ا


In [15]:
sequences = char_dataset.batch(101, drop_remainder=True)
sequences

<BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int32, name=None)>

In [16]:
for i in sequences.take(3):
    print('--->', ''.join(index2char[i.numpy()]))

---> |برخیز بتا بیا ز بهر دل ما
|حل کن به جمال خویشتن مشکل ما
|یک کوزه شراب تا به هم نوش کنیم
|زآن پیش که 
---> کوزه ها کنند از گل ما
|چون عهده نمی شود کسی فردا را
|حالی خوش دار این دل پر سودا را
|می نوش به ماهتاب
--->  ای ماه که ماه
|بسیار بتابد و نیابد ما را
|قرآن که مهین کلام خوانند آن را
|گهگاه نه بر دوام خوانند آن


In [17]:
def sit(batch):
    input_text = batch[:-1]
    target_text = batch[1:]
    return input_text, target_text
dataset = sequences.map(sit)

In [18]:
dataset

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>

In [19]:
for i in dataset.take(1):
    print(''.join(index2char[i[0].numpy()]))
    print(''.join(index2char[i[1].numpy()]))

|برخیز بتا بیا ز بهر دل ما
|حل کن به جمال خویشتن مشکل ما
|یک کوزه شراب تا به هم نوش کنیم
|زآن پیش که
برخیز بتا بیا ز بهر دل ما
|حل کن به جمال خویشتن مشکل ما
|یک کوزه شراب تا به هم نوش کنیم
|زآن پیش که 


In [20]:
dataset = dataset.batch(64, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [21]:
len(vocabolaries)

53

In [22]:
vocabolary_size = len(vocabolaries)
embedding_dim = 256
rnn_unit = 1024

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabolary_size, embedding_dim),
    tf.keras.layers.GRU(rnn_unit, return_sequences=True),
    tf.keras.layers.Dense(vocabolary_size)
])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         13568     
                                                                 
 gru (GRU)                   (None, None, 1024)        3938304   
                                                                 
 dense (Dense)               (None, None, 53)          54325     
                                                                 
Total params: 4,006,197
Trainable params: 4,006,197
Non-trainable params: 0
_________________________________________________________________


In [25]:
for input_text, target_text in dataset.take(1):
    output = model.predict(input_text)
    print(output[0])

2/2 [==============================] - 1s 217ms/step
[[ 3.9367056e-03 -4.3200562e-05 -7.8365784e-03 ...  2.6279837e-03
  -9.9309478e-03  7.9644928e-03]
 [ 4.0994259e-05 -4.0138853e-03  5.6392979e-03 ... -2.4955720e-05
  -1.1167035e-02 -6.6039660e-03]
 [-4.3606907e-03  1.8664117e-03 -2.4380321e-03 ... -1.0182662e-02
  -2.8464333e-03 -8.0791246e-03]
 ...
 [ 1.1493186e-03 -1.4508415e-02  2.7523055e-03 ... -7.3470417e-03
  -8.3011054e-03 -7.9060812e-03]
 [-8.2391892e-03 -1.9515365e-02  5.0249491e-03 ... -8.5978433e-03
  -7.6330556e-03 -1.3436531e-03]
 [-6.8588904e-04 -6.2629483e-03 -2.2095209e-04 ... -3.7221555e-03
   8.9248037e-04  1.9277334e-03]]


In [26]:
si = tf.random.categorical(output[0], num_samples=1)
si

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[13],
       [ 4],
       [25],
       [31],
       [46],
       [41],
       [18],
       [20],
       [42],
       [31],
       [34],
       [27],
       [ 6],
       [ 1],
       [27],
       [ 4],
       [12],
       [50],
       [45],
       [33],
       [15],
       [29],
       [22],
       [ 7],
       [ 5],
       [ 4],
       [37],
       [ 6],
       [38],
       [ 8],
       [52],
       [49],
       [52],
       [37],
       [21],
       [ 8],
       [13],
       [28],
       [45],
       [32],
       [39],
       [ 9],
       [10],
       [39],
       [16],
       [ 0],
       [43],
       [19],
       [ 3],
       [15],
       [23],
       [13],
       [45],
       [ 5],
       [50],
       [38],
       [38],
       [49],
       [27],
       [15],
       [25],
       [28],
       [19],
       [11],
       [46],
       [28],
       [ 3],
       [23],
       [40],
       [28],
       [49],
       [35],
       [33],
   

In [27]:
tf.squeeze(si, axis=-1).numpy()

array([13,  4, 25, 31, 46, 41, 18, 20, 42, 31, 34, 27,  6,  1, 27,  4, 12,
       50, 45, 33, 15, 29, 22,  7,  5,  4, 37,  6, 38,  8, 52, 49, 52, 37,
       21,  8, 13, 28, 45, 32, 39,  9, 10, 39, 16,  0, 43, 19,  3, 15, 23,
       13, 45,  5, 50, 38, 38, 49, 27, 15, 25, 28, 19, 11, 46, 28,  3, 23,
       40, 28, 49, 35, 33, 47, 47,  6, 30, 43, 21,  0, 47,  8,  2, 43, 40,
       34, 50, 13,  0,  1, 26, 22,  4, 13, 25, 43, 13,  0, 30, 44],
      dtype=int64)

In [28]:
''.join(index2char[tf.squeeze(si, axis=-1).numpy()])

'ب\xa0شغپَحدُغلض؟ ض\xa0اگٔقتظرء،\xa0ه؟وآیکیهذآبطٔفٌأؤٌث\nِخ|تزبٔ،گووکضتشطخئپط|زٍطکمقچچ؟عِذ\nچآ!ٍِلگب\n صر\xa0بشِب\nعّ'

In [29]:
output[0][0]

array([ 3.9367056e-03, -4.3200562e-05, -7.8365784e-03, -1.9283213e-03,
       -1.3061013e-02, -1.5834630e-02, -1.2856413e-02,  4.3911971e-03,
       -2.8877882e-03, -3.0295765e-03, -7.3078480e-03,  3.3625020e-03,
       -1.5630694e-03, -9.1980379e-03,  1.0850260e-02, -3.7533219e-03,
        5.5655302e-03,  8.3686076e-03, -6.6745048e-03, -9.5991120e-03,
        9.9801254e-03,  7.3177014e-03, -1.2245760e-02, -2.1555028e-03,
       -7.1079703e-06,  1.2962503e-03,  7.1030180e-03,  3.3082997e-03,
        3.2347909e-03,  2.8928085e-03, -9.4452333e-03,  3.0014114e-03,
        1.1288361e-03,  1.0684471e-02, -4.2153518e-03, -8.5985586e-03,
       -1.6348558e-02, -2.7941749e-03, -1.1262357e-02, -9.1878008e-03,
        7.0311134e-03,  5.3856242e-03,  1.2381913e-02, -2.1191482e-03,
       -7.7927411e-03,  1.1801550e-03, -1.2002817e-02,  2.9044212e-03,
       -9.0663237e-03,  1.3040710e-02,  2.6279837e-03, -9.9309478e-03,
        7.9644928e-03], dtype=float32)

In [30]:
def loss_f(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss_f)

In [31]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='khayyammoulavi/checkpoints', save_weights_only=True)

In [32]:
# history = model.fit(dataset, epochs=100, callbacks=[checkpoint]) #loss: 0.0548

In [33]:
tf.train.latest_checkpoint('khayyammoulavi')

'khayyammoulavi\\checkpoints'

In [34]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabolary_size, embedding_dim),
    tf.keras.layers.GRU(rnn_unit, return_sequences=True),
    tf.keras.layers.Dense(vocabolary_size)
])

In [35]:
model_2.load_weights(tf.train.latest_checkpoint('khayyammoulavi'))

In [36]:
model_2.build(tf.TensorShape([1, None]))

In [37]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 256)         13568     
                                                                 
 gru_1 (GRU)                 (None, None, 1024)        3938304   
                                                                 
 dense_1 (Dense)             (None, None, 53)          54325     
                                                                 
Total params: 4,006,197
Trainable params: 4,006,197
Non-trainable params: 0
_________________________________________________________________


In [38]:
first_string = 'به نام خداوند جان و خرد'
input_eval = [char2index[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)
input_eval

<tf.Tensor: shape=(1, 23), dtype=int32, numpy=
array([[13, 37,  1, 36, 12, 35,  1, 19, 20, 12, 38, 36, 20,  1, 17, 12,
        36,  1, 38,  1, 19, 22, 20]])>

In [39]:
model_2.reset_states()

In [40]:
from tqdm.notebook import tqdm

In [41]:
text_generated = ['به نام خداوند جان و خرد']
for i in tqdm(range(500)):
    predictions = model_2.predict(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predicted_ids = np.array(predictions.numpy()).argmax(axis=1).reshape(-1, 1)[-1][0]
    message = np.append(input_eval[0].numpy(), predicted_ids)[1:]
    input_eval = tf.expand_dims(message, 0)
    text_generated.append(index2char[predicted_ids])

  0%|          | 0/500 [00:00<?, ?it/s]

1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 47ms/step


In [42]:
''.join(text_generated).split('\n')

['به نام خداوند جان و خردمن',
 '|زیرا که هر خواهد و رفتن بست',
 '|چنانکه آو خواهی که بدی',
 '|این کوزه گر دهر چنین جام لطیف',
 '|می سازد و باز بر زمین می شود',
 '|یک پاره گوهر می شود یک پاره لعل و کهربا',
 '|ای طالب دیدار او بنگر تشقه هست مرا',
 '|هر کس به جزار بود خام غمان چون مهر در خانه نو',
 '|زین و ده چه باد چون گردد دیگر فد',
 '|آن که آید به دست و پای پارسا',
 '|در مرگ هشیاری نهی در برج مهر و کین تو',
 '|ز رنگ اندیشه ز هم دم سر او در رهنا',
 '|بینم مه نه عالم مست و خرا',
 '|با لب و شهوت باش و با سیم تنی',
 '|کانکس که جهان کرد فراغت دارد',
 '|از سبلت چون تویی و ریش از ما',
 '|ج']

In [43]:
for i in ''.join(text_generated).split('\n'):
    print(i)

به نام خداوند جان و خردمن
|زیرا که هر خواهد و رفتن بست
|چنانکه آو خواهی که بدی
|این کوزه گر دهر چنین جام لطیف
|می سازد و باز بر زمین می شود
|یک پاره گوهر می شود یک پاره لعل و کهربا
|ای طالب دیدار او بنگر تشقه هست مرا
|هر کس به جزار بود خام غمان چون مهر در خانه نو
|زین و ده چه باد چون گردد دیگر فد
|آن که آید به دست و پای پارسا
|در مرگ هشیاری نهی در برج مهر و کین تو
|ز رنگ اندیشه ز هم دم سر او در رهنا
|بینم مه نه عالم مست و خرا
|با لب و شهوت باش و با سیم تنی
|کانکس که جهان کرد فراغت دارد
|از سبلت چون تویی و ریش از ما
|ج
